# CSCI 3202, Spring 2018

# Friday 23 February 2018

# In-class notebook:  Games

<a id='top'></a>

<br>

### Your name(s):

<br>

* When you submit this to Moodle (under Quizlet 7), be sure to include all of your group members' names.
* You may work in groups of up to 3 people,
* but **all people** in the group must submit the assignment on their own Moodle account (because Moodle is a pain in the ass to create groups and this will still be faster than your normal quizlets).

---

Shortcuts:  [Top](#top) || [Board](#board) | [Players](#players) | [Utility](#ulitity) | [Playing Games](#playing) | [TTT Class](#class) || [Bottom](#bottom)

---

Before we begin, let's load a few packages that we might find useful.

In [17]:
import numpy as np
import copy as cp

<br>

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>


# Tic-Tac-Toe

Today, the goal is to get an operational `TicTacToe` class, and - aspirationally - play games using `random_player`s.  We know that if many, many games are played between two random players, the one who goes first should win about 58% of the time, so that is our benchmark... once we get there.

Let's start by defining a Tic-Tac-Toe board `State` class.  This class includes everything we need to know about the board:
* whose move is it? (`to_move`)
* has anyone won yet? (`utility`)
* what marks are on the board? (`board`, a dictionary that we can modify so the keys are tuples representing a location on the board, and the values are the player, 'X' or 'O', occupying those locations)
* what moves are left to be made? (`moves`, a list of tuples representing vacant board locations)

In [18]:
class State:
    def __init__(self, moves):
        self.to_move = 'O'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)

Note that we used the `copy` method so that when we modify what moves are left, we do not tamper with the original `moves` object in memory.

Have a look at the image above the "Tic-Tac-Toe" header.  How would we represent that state?

Well, of course we need to construct a `State` first.  You'll notice that in order to do that, we need the list of initially available `moves`.  Let's use row/column (matrix-esque) notation.  So (1,1) represents the O in the upper-left, there is another O at (2,2) in the center square, and X's are in (1,3), (3,1) and (3,3).

Armed with this notation, the list of initially available moves is:

In [19]:
moves_initial = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3), (3,1), (3,2), (3,3)]

That was nice and all, but really we ought to generalize this so we are not hard-coded for the $3 \times 3$ Tic-Tac-Toe case.  See if you can figure out a way to initialize the list of initially available `moves` for an arbitrary sized $m \times n$ Tic-Tac-Toe board.  (If you can't, that's okay, you can just leave `moves_initial` alone and move on to the rest of this notebook.)

In [20]:
nrow = 3
ncol = 3
moves = []
def create_moves(nrow, ncol):
    moves = []
    for i in range(1, nrow+1):
        for j in range(1, ncol+1):
            moves.append((i,j))
    return moves
print(create_moves(nrow, ncol))
# your code goes here!



[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]


<br>

[Back to top](#top)
<a/ id='board'></a>

### Board and game state

Now we need to modify the board.  Each time a player makes a mark on the board, one of the potential moves has been taken and must be removed from the list, and we need to update whose move it is.  I will add the first move - you do the rest!

In [21]:
state = State(moves_initial)
move = (1,3)
state.board[move] = 'X'                             # player makes a move
state.moves.remove(move)                            # that move is no longer valid
state.to_move = 'O' if state.to_move=='X' else 'X'  # now it is the next player's turn


# make the other moves down here!



Since it is awfully tedious to keep re-running the same snippet of code, let's define a function to do it for us.

In [22]:
def result(move, state):
    '''
    What is the hypothetical result of move `move` in state `state` ?
    move  = (row, col) tuple where player will put their mark (X or O)
    state = a `State` object, to represent whose turn it is and form
            the basis for generating a **hypothetical** updated state
            that will result from making the given `move`
    '''
    # First, create a copy of the given state. We use `deepcopy` so
    # that we do not prematurely tamper with the actual current 
    # Tic-Tac-Toe state, since we mostly use this method
    # to get hypothetical results as we search our game tree, not
    # for actual moves.
    new_state = cp.deepcopy(state)
    # player makes a move
    
    new_state.moves.remove(move)                            # that move is no longer valid
    new_state.to_move = 'O' if new_state.to_move=='X' else 'X'  # now it is the next player's turn
    new_state.board[move] = new_state.to_move

    
    return new_state

Let's take our new function for a test drive by re-creating the state in the above picture.  Note that as long as I alternate X and O, the order of the moves is arbitrary (since we don't know in what order that image was created).

In [23]:
state = State(moves_initial)
state = result((1,3), state)
state = result((1,1), state)
state = result((3,1), state)
state = result((2,2), state)
state = result((3,3), state)

Verify that we get the expected results out of the new `state`:
* whose turn should it be?
* what moves are left?
* what does the board look like?

In [24]:
print(state.board)

{(1, 3): 'X', (1, 1): 'O', (3, 1): 'X', (2, 2): 'O', (3, 3): 'X'}


It sure would be easier to verify that the proper state had been generated if we had an appropriate helper function to visualize the Tic-Tac-Toe board as it appears in normal life, instead of as a dictionary.  Why, here's one now!  Test it out on the `state` you just generated above.  Note also that this is hard-coded for the $3\times 3$ case currently.

In [40]:
def display(state):
    list1 = []
    for row in range(1, 4):
        for col in range(1, 4):
            #print(list(state.board.keys())[list(state.board.values()).index('X')])
            print(state.board.get((row, col), '.'), end=' ')
            if state.board.get((row,col)) == 'X':
                list1.append((row,col))
        print(list1)

In [41]:
display(state)

O . X [(1, 3)]
. O . [(1, 3)]
X . X [(1, 3), (3, 1), (3, 3)]


<br>
[Back to top](#top)
<a/ id='players'></a>

### Players

Now let's create a player to play a random legal move in a given Tic-Tac-Toe game state.

In [27]:
def random_player(state):
    '''A player that chooses a legal move at random out of all
    available legal moves in Tic-Tac-Toe state argument'''

    
    # your code goes here...

    
    return random_move

<a/ id='utility'></a>

### Utility

Now a game between two random players can be represented as a loop until either all available moves are taken up (`length(state.moves)==0`) ***or*** one of the players gets 3-in-a-row.  So now we need a function to check whether or not someone has achieved 3-in-a-row by making `move` in `state`.  I've given you some guidelines on how to return the computed `utility`, but there will be some modifications needed depending on how you implement this routine.

<img src="https://imgs.xkcd.com/comics/will_it_work.png" width="230"/>

A couple notes:
* For purposes of getting off the ground during class, there are only 8 possible ways in standard $3 \times 3$ Tic-Tac-Toe in which a player can win.  So it isn't too difficult to hard-code for this situation, if you are stumped. An example portion of this hard-coding solution is given as a comment below.
* You only need to check if the person who is making the `move` has just gotten 3-in-a-row, because the other player can't possibly have changed the utility since they have not moved.
* Remember that `compute_utility` is a **hypothetical** calculation.  We do **not** want to actually modify `state` yet because once we start using alpha-beta pruning, we will need to make judgments on how good numerous possible outcomes are *before* we execute our move and modify the state.
  * To do this, we can use a `deepcopy` of the state board, in which the new board object itself is essentially a new version of the old one and not attached in memory at all.  This line is given for you.

In [28]:
def compute_utility(move, state):
    '''Calculate the utility of making a given move in a given state.
    If X wins, utility is +1; if O wins, utility is -1; if it is a
    draw or nobody has won yet, then utility is 0.'''
    
    # create a hypothetical copy of the board, with 'move' executed
    board = cp.deepcopy(state.board)
    board[move] = state.to_move

    # Example hard-coding line:
    # if (state.board.get((1,1))==state.to_move and
    #     state.board.get((1,2))==state.to_move and
    #     state.board.get((1,3))==state.to_move) or
    # ... continue with the other ways in which someone could win...
    
   
    # your code goes here
    

    if # nobody has gotten 3-in-a-row
        return 0
    else:
        return 1 if state.to_move=='X' else -1

SyntaxError: invalid syntax (<ipython-input-28-408d17b33ccd>, line 20)

In [58]:
horizontal = []
vertical = []
diagonal = []
for i in range(1, 4):
    for j in range(1, 4):
        horizontal.append((j,i))
        vertical.append((i,j))
for i in range(1,4):
    diagonal.append((i,i))
#print(horizontal)
#print(vertical)
#print(diagonal)
import itertools
top5 = list(itertools.islice(horizontal, 5)) # grab the first five elements
#print(top5)

top3 = set(itertools.islice(horizontal, 3))
print(top3)
n = 3
print(horizontal)
del horizontal[:n]
print(horizontal)


{(3, 1), (1, 1), (2, 1)}
[(1, 1), (2, 1), (3, 1), (1, 2), (2, 2), (3, 2), (1, 3), (2, 3), (3, 3)]
[(1, 2), (2, 2), (3, 2), (1, 3), (2, 3), (3, 3)]


In [59]:
temp = []
win_condition = [(1,1), (2,2), (3,3)]
win_condition = set(win_condition)
horizontal = set(horizontal)
if win_condition.issubset(horizontal):
    print('1')


We can test out our `compute_utility` method by having O make a move in the (1,2) square, and then checking if someone has won yet:

In [60]:
move = (1,2)

# the utility is a *hypothetical* calculation
new_utility = compute_utility(move, state)

# make the actual state update
state = result(move, state)
state.utility = new_utility

# we should stop playing our game once someone has won (i.e., utility != 0)
print('Has someone won yet? {}'.format(state.utility!=0))

NameError: name 'compute_utility' is not defined

It seems like updating the utility via `compute_utility` really ought to be a part of the `result` function, since the new utility is a direct result of the move.  Go back to add this to your `result` routine.

Now then.  What is the utility if we have X place its next mark at (2,3)?

In [61]:
move = (2,3)
state = result(move, state)

# we should stop playing our game once someone has won (i.e., utility != 0)
print('Has someone won yet? {}'.format(state.utility!=0))

Has someone won yet? False


You should have just found out that X has emerged from this heated battle triumphant!  Glorious victory for X!

Now it's time to define a function to play a game between two players. The function should take as arguments the function names for two players (so far, all we have are `random_player`s), and return the final state of the game.
* Start by initializing a new game `State`
* Alternate between players selecting a move and then applying that move to modify the game `state`
* After each player's move, you should have a check to see if the game is over.
* If the game is over, print to the screen the `state.board` and `state.utility`.

In [62]:
def play_game(player1, player2):


SyntaxError: unexpected EOF while parsing (<ipython-input-62-c7368c4de038>, line 1)

<br>

<a/ id='playing'></a>
[Back to top](#top)

### Let's play a game!

Play a game of Tic-Tac-Toe between two random players. Let's see what happens...

In [63]:
state = play_game(random_player, random_player)
display(state)

NameError: name 'play_game' is not defined

### Okay... let's play LOTS OF GAMES!

Now create a loop to play 1000 games of random vs random Tic-Tac-Toe.  Store the resulting utility of each match.  Remember, we are keeping track of the wins/losses of the player who goes first.  Check to making sure that this player's winning percentage is around 58%.

<br>

<a/ id='class'></a>
[Back to top](#top)

### A Tic-Tac-Toe class - where we're going with all this

When we apply alpha-beta pruning to Tic-Tac-Toe, we will want to have a nice compact class description for a game of Tic-Tac-Toe.  We already have the game `State` defined above.  But now we should define a class with methods.  This will avoid hard-coding things like the size of the board, how many marks in a row a player needs to win, and even internalizes the game state within a game class object.

Luckily, you already have defined most of the methods we need:
* `result(self, move, state)`
* `compute_utility(self, move, state)`
* `game_over(self, state)` -- this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs
* `utility(self, state, player)` -- this one also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \times$ utility if the player is O.
* `display(self)`
* `play_game(self, player1, player2)`

If you have time, you will be well-served on your homework to start filling in the class definition.  Then, use it to re-create the 58% benchmark test for the long-run frequency of first-player wins.

In [64]:
class TicTacToe:
    def __init__(self, nrow=3, ncol=3, nwin=3):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = # insert your general list of nrow x ncol moves here
        self.state = State(moves)

    def result(self, move, state):

    def compute_utility(self, move, state):

    def game_over(self, state):

    def utility(self, state, player):

    def display(self):

    def play_game(self, player1, player2):


SyntaxError: invalid syntax (<ipython-input-64-5d52bd661d90>, line 6)

<a/ id='bottom'></a>
[Back to top](#top)